In [11]:
import pandas as pd
import html
import xml.etree.ElementTree as ET

In [33]:
df_error = pd.read_excel('../do_not_commit/Events/ApplicationEvents.xlsx', sheet_name='ApplicationError')
df_hang = pd.read_excel('../do_not_commit/Events/ApplicationEvents.xlsx', sheet_name='ApplicationHang')
df = pd.concat([df_error, df_hang]).reset_index()

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117 entries, 0 to 116
Data columns (total 25 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   index                                 117 non-null    int64         
 1   RWB_EVENTRAWRESULTITEM_ID             117 non-null    int64         
 2   RWB_ETL_EVENT_DESTINATION_IDENTIFIER  117 non-null    int64         
 3   RWB_CREATE_TIMESTAMP                  117 non-null    object        
 4   RWB_EFFECTIVE_DATE                    117 non-null    datetime64[ns]
 5   ID                                    117 non-null    float64       
 6   ClientItemKey                         117 non-null    int64         
 7   LogName                               0 non-null      float64       
 8   ProviderName                          117 non-null    object        
 9   ProviderGUID                          117 non-null    object        
 10  Ev


For XML structures of type 'error', map them in Python from the following SQL query:


select ID
, ClientItemKey
, TimeCreatedSystemTime
, User_Name0
, Netbios_Name0
, SerialNumber
, User_Principal_Name0
, Bias0
, DaylightBias0
, Description0
, CurrentTimeZone0
, Local_Time,
/\* Values to pull \*/
FaultingApplicationName= Coalesce(f_XML.value('(/EventDataXml/EventData/Data/node())[1]', 'nvarchar(max)'),'Not Available'),
AppVersion= f_XML.value('(/EventDataXml/EventData/Data/node())[2]', 'nvarchar(max)'),
AppTimestamp= f_XML.value('(/EventDataXml/EventData/Data/node())[3]', 'nvarchar(max)'),
FaultingModuleName= f_XML.value('(/EventDataXml/EventData/Data/node())[4]', 'nvarchar(max)'),
ModuleVersion= f_XML.value('(/EventDataXml/EventData/Data/node())[5]', 'nvarchar(max)'),
ModuleTimestamp= f_XML.value('(/EventDataXml/EventData/Data/node())[6]', 'nvarchar(max)'),
ExceptionCode= f_XML.value('(/EventDataXml/EventData/Data/node())[7]', 'nvarchar(max)'),
FaultOffset= f_XML.value('(/EventDataXml/EventData/Data/node())[8]', 'nvarchar(max)'),
FaultingProcessId= f_XML.value('(/EventDataXml/EventData/Data/node())[9]', 'nvarchar(max)'),
FaultingApplicationStartTime= f_XML.value('(/EventDataXml/EventData/Data/node())[10]', 'nvarchar(max)'),
FaultingApplicationPath= f_XML.value('(/EventDataXml/EventData/Data/node())[11]', 'nvarchar(max)'),
FaultingApplicationModulePath= f_XML.value('(/EventDataXml/EventData/Data/node())[12]', 'nvarchar(max)'),
ReportId= f_XML.value('(/EventDataXml/EventData/Data/node())[13]', 'nvarchar(max)'),
FaultingPackageFullName= f_XML.value('(/EventDataXml/EventData/Data/node())[14]', 'nvarchar(max)'),
FaultingPackageRelativeApplicationID= f_XML.value('(/EventDataXml/EventData/Data/node())[15]', 'nvarchar(max)'),
/\* Custom Data \*/
FullPath= f_XML.value('(/EventDataXml/CustomData/FullPath/node())[1]', 'nvarchar(max)'),
ProductVersion= f_XML.value('(/EventDataXml/CustomData/ProductVersion/node())[1]', 'nvarchar(max)'),
ProductName= Coalesce(f_XML.value('(/EventDataXml/CustomData/ProductName/node())[1]', 'nvarchar(max)'),'Not Available'),
Publisher= f_XML.value('(/EventDataXml/CustomData/Publisher/node())[1]', 'nvarchar(max)'),
ProgramId= f_XML.value('(/EventDataXml/CustomData/ProgramId/node())[1]', 'nvarchar(max)'),
FileId= f_XML.value('(/EventDataXml/CustomData/FileId/node())[1]', 'nvarchar(max)'),
FileVersion= f_XML.value('(/EventDataXml/CustomData/FileVersion/node())[1]', 'nvarchar(max)')
from
(
    SELECT ID, ClientItemKey, TimeCreatedSystemTime,
Cast(Replace(Replace(Replace(Replace(Replace(Replace(CAST(EventDataXML as nvarchar(MAX)),' xmlns="http://schemas.microsoft.com/win/2004/08/events/event"','') ,'&gt;&lt;','>blank<'),'&lt;', '<'),'&gt;','>'),'<![CDATA[',''),']]>','') as XML)  as f_XML
, s.User_Name0
, s.Netbios_Name0
, s.SerialNumber
, USR.User_Principal_Name0
, GST.Bias0
, GST.DaylightBias0
, GST.Description0
, GCS.CurrentTimeZone0
, DATEADD(MI,GCS.CurrentTimeZone0,e.TimeCreatedSystemTime) as Local_Time
From
v_ClientEvents as e
Left Outer JOIN v_R_System s on e.ClientItemKey = s.ResourceID
Left Outer join v_GS_TIME_ZONE GST on e.ClientItemKey = GST.ResourceID
Left Outer Join v_GS_Computer_System GCS on e.ClientItemKey = GCS.ResourceID
LEFT OUTER JOIN v_R_User USR ON s.User_Name0 = USR.User_Name0
Where
ProviderName = 'Application Error'
And TimeCreatedSystemTime >= GetDate()-10) as base

In [102]:
def xml_to_col(xml_str, provider_name):

    # Extract CDATA section
    data_string = xml_str
    unescaped_string = html.unescape(data_string)

    # Extract CDATA section
    start = unescaped_string.find('<![CDATA[') + len('<![CDATA[')
    end = unescaped_string.find(']]>')
    cdata = unescaped_string[start:end]

    # Wrap the CDATA content with a root element
    wrapped_cdata = '<root>' + cdata + '</root>'
    wrapped_cdata

    # Parse the wrapped CDATA as XML
    root = ET.fromstring(wrapped_cdata)
    root[0][-1].text

    if provider_name == 'error':
        row_dict = {
            'FaultingApplicationName':'',
            'AppVersion': '',
            'AppTimestamp': '',
            'FaultingModuleName': '',
            'ModuleVersion': '',
            'ModuleTimestamp': '',
            'ExceptionCode': '',
            'FaultOffset': '',
            'FaultingProcessId': '',
            'FaultingApplicationStartTime': '',
            'FaultingApplicationPath': '',
            'FaultingApplicationModulePath': '',
            'ReportId': '',
            'FaultingPackageFullName': '',
            'FaultingPackageRelativeApplicationID': '',
            'FullPath': '',
            'ProductVersion': '',
            'ProductName': '',
            'Publisher': '',
            'ProgramId': '',
            'FileId': '',
            'FileVersion': ''
        }
    elif provider_name == 'hang':
        row_dict = {
            'Program':'',
            'ProgramVersion': '',
            'ProcessID': '',
            'StartTime': '',
            'TerminationType': '',
            'ApplicationPath': '',
            'ReportID': '',
            'FaultingPackageFullName': '',
            'FaultingPackageRelativeApplicationID': '',
            'HangType': '',
            'BinaryValue': '',
            'FullPath': '',
            'ProductVersion': '',
            'ProductName': '',
            'Publisher': '',
            'ProgramId': '',
            'FileId': '',
            'FileVersion': '',
        }

    for i in range(len(root[0])):
        row_dict[list(row_dict.items())[i][0]] = root[0][i].text
    for i in range(len(root[1])):
        if i < 7:
            row_dict[list(row_dict.items())[int(i + len(root[0]))][0]] = root[1][i].text
    return row_dict

In [71]:
xml_to_col(df['EventDataXML'][99], 'hang')

{'Program': 'svchost.exe',
 'ProgramVersion': '10.0.19041.1806',
 'ProcessID': '1770',
 'StartTime': '01d9b578a90fb1c7',
 'TerminationType': '4294967295',
 'ApplicationPath': 'C:\\Windows\\System32\\svchost.exe',
 'ReportID': 'bc40b853-5688-4b83-9116-f5072d1107be',
 'FaultingPackageFullName': None,
 'FaultingPackageRelativeApplicationID': None,
 'HangType': 'Cross-process',
 'BinaryValue': '430072006F00730073002D00700072006F00630065007300730000000000',
 'FullPath': 'C:\\Windows\\System32\\svchost.exe',
 'ProductVersion': '10.0.19041.1',
 'ProductName': 'Microsoft® Windows® Operating System',
 'Publisher': 'Microsoft Corporation',
 'ProgramId': '0000F8FE4DDB12E22396DB24B977FFEC15D500000000',
 'FileId': '0000DFB8FE6C32CFB708B42EFBFF978DF011C5C46CD9',
 'FileVersion': '10.0.19041.1 (WinBuild.160101.0800)'}

In [99]:
# RJ 07/17/2023: There are 3 extra attributes in this particular XML:
# FaultBucket, WatsonEventName, UserImpactVector.
data_string = df.loc[98, 'EventDataXML']
unescaped_string = html.unescape(data_string)

# Extract CDATA section
start = unescaped_string.find('<![CDATA[') + len('<![CDATA[')
end = unescaped_string.find(']]>')
cdata = unescaped_string[start:end]

# Wrap the CDATA content with a root element
wrapped_cdata = '<root>' + cdata + '</root>'
wrapped_cdata

# Parse the wrapped CDATA as XML
root = ET.fromstring(wrapped_cdata)
# root[1][10].text
wrapped_cdata

'<root><EventData xmlns="http://schemas.microsoft.com/win/2004/08/events/event">\n  <Data>ScreenSketch.exe</Data>\n  <Data>10.2008.3001.0</Data>\n  <Data>2570</Data>\n  <Data>01d9b41456e9152e</Data>\n  <Data>4294967295</Data>\n  <Data>C:\\Program Files\\WindowsApps\\Microsoft.ScreenSketch_10.2008.3001.0_x64__8wekyb3d8bbwe\\ScreenSketch.exe</Data>\n  <Data>2203e174-1c10-4d3e-9074-af1e8edc2ac1</Data>\n  <Data>Microsoft.ScreenSketch_10.2008.3001.0_x64__8wekyb3d8bbwe</Data>\n  <Data>App</Data>\n  <Data>Quiesce</Data>\n  <Binary>510075006900650073006300650000000000</Binary>\n</EventData>\n<CustomData>\n  <FullPath>C:\\Program Files\\WindowsApps\\Microsoft.ScreenSketch_10.2008.3001.0_x64__8wekyb3d8bbwe\\ScreenSketch.exe</FullPath>\n  <ProductVersion>10.2008.3001.0</ProductVersion>\n  <ProductName>Microsoft Snip &amp; Sketch</ProductName>\n  <Publisher>Microsoft Corporation</Publisher>\n  <ProgramId>0000A2EE476710D0F769B37456265182CE5500000000</ProgramId>\n  <FileId>0000A9607B08C2E511986C53F5

In [103]:
xml_to_col(df['EventDataXML'][98], 'hang')

{'Program': 'ScreenSketch.exe',
 'ProgramVersion': '10.2008.3001.0',
 'ProcessID': '2570',
 'StartTime': '01d9b41456e9152e',
 'TerminationType': '4294967295',
 'ApplicationPath': 'C:\\Program Files\\WindowsApps\\Microsoft.ScreenSketch_10.2008.3001.0_x64__8wekyb3d8bbwe\\ScreenSketch.exe',
 'ReportID': '2203e174-1c10-4d3e-9074-af1e8edc2ac1',
 'FaultingPackageFullName': 'Microsoft.ScreenSketch_10.2008.3001.0_x64__8wekyb3d8bbwe',
 'FaultingPackageRelativeApplicationID': 'App',
 'HangType': 'Quiesce',
 'BinaryValue': '510075006900650073006300650000000000',
 'FullPath': 'C:\\Program Files\\WindowsApps\\Microsoft.ScreenSketch_10.2008.3001.0_x64__8wekyb3d8bbwe\\ScreenSketch.exe',
 'ProductVersion': '10.2008.3001.0',
 'ProductName': 'Microsoft Snip & Sketch',
 'Publisher': 'Microsoft Corporation',
 'ProgramId': '0000A2EE476710D0F769B37456265182CE5500000000',
 'FileId': '0000A9607B08C2E511986C53F5505ACAEF38F365D66A',
 'FileVersion': '10.2008.3001.0'}

In [104]:
for index, row in df.iterrows():
    print(index)
    if row['ProviderName'] == 'Application Error':
        for column, value in xml_to_col(row['EventDataXML'], 'error').items():
            df.loc[index, column] = value
    elif row['ProviderName'] == 'Application Hang':
        for column, value in xml_to_col(row['EventDataXML'], 'hang').items():
            print(row['EventDataXML'])
            df.loc[index, column] = value

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
<EventDataXml>&lt;![CDATA[&lt;EventData xmlns="http://schemas.microsoft.com/win/2004/08/events/event"&gt;
  &lt;Data&gt;ScreenSketch.exe&lt;/Data&gt;
  &lt;Data&gt;10.2008.3001.0&lt;/Data&gt;
  &lt;Data&gt;1eb8&lt;/Data&gt;
  &lt;Data&gt;01d9b6516e2aeae8&lt;/Data&gt;
  &lt;Data&gt;4294967295&lt;/Data&gt;
  &lt;Data&gt;C:\Program Files\WindowsApps\Microsoft.ScreenSketch_10.2008.3001.0_x64__8wekyb3d8bbwe\ScreenSketch.exe&lt;/Data&gt;
  &lt;Data&gt;33f023b8-09c7-4d93-9bda-ba1a734f5902&lt;/Data&gt;
  &lt;Data&gt;Microsoft.ScreenSketch_10.2008.3001.0_x64__8wekyb3d8bbwe&lt;/Data&gt;
  &lt;Data&gt;App&lt;/Data&gt;
  &lt;Data&gt;Quiesce&lt;/Data&gt;
  &lt;Binary&gt;510075006900650073006300650000000000&lt;/Binary&gt;
&lt;/EventData&gt;
&lt;CustomData&gt;
  &lt;FullPath&gt;C:\Program Files\WindowsApps\Microsoft.ScreenSketch_10.2

In [105]:
df

,index,RWB_EVENTRAWRESULTITEM_ID,RWB_ETL_EVENT_DESTINATION_IDENTIFIER,RWB_CREATE_TIMESTAMP,RWB_EFFECTIVE_DATE,ID,ClientItemKey,LogName,ProviderName,ProviderGUID,...,FileVersion,Program,ProgramVersion,ProcessID,StartTime,TerminationType,ApplicationPath,ReportID,HangType,BinaryValue
0,0,10595750,11627,2023-07-15 08:46:26 -05:00,2023-07-15,7.205759e+16,16808298,NaN,Application Error,{00000000-0000-0000-0000-00000000000,...,16.0.16327.20324,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,10595751,11627,2023-07-15 08:46:26 -05:00,2023-07-15,7.205759e+16,16808298,NaN,Application Error,{00000000-0000-0000-0000-00000000000,...,16.0.16327.20324,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,10609965,11627,2023-07-15 08:46:26 -05:00,2023-07-15,7.205759e+16,16808298,NaN,Application Error,{00000000-0000-0000-0000-00000000000,...,7.34.0.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,10609966,11627,2023-07-15 08:46:26 -05:00,2023-07-15,7.205759e+16,16808298,NaN,Application Error,{00000000-0000-0000-0000-00000000000,...,"81,041,0000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,10609967,11627,2023-07-15 08:46:26 -05:00,2023-07-15,7.205759e+16,16808298,NaN,Application Error,{00000000-0000-0000-0000-00000000000,...,7.34.0.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,53,10458228,11557,2023-07-14 08:46:34 -05:00,2023-07-14,7.205759e+16,16807236,NaN,Application Hang,{00000000-0000-0000-0000-00000000000,...,10.0.19041.2193 (WinBuild.160101.0800),LockApp.exe,10.0.19041.2193,1ad8,01d9b4d216833c4e,4294967295,C:\Windows\SystemApps\Microsoft.LockApp_cw5n1h...,e5c6683b-cfc2-4531-8095-7258a715fe16,Activation,410063007400690076006100740069006F006E0000000000
113,54,10458265,11557,2023-07-14 08:46:34 -05:00,2023-07-14,7.205759e+16,16814160,NaN,Application Hang,{00000000-0000-0000-0000-00000000000,...,10.0.19041.2193 (WinBuild.160101.0800),LockApp.exe,10.0.19041.2193,669c,01d9b5a68ab3c8d3,4294967295,C:\Windows\SystemApps\Microsoft.LockApp_cw5n1h...,2cb14167-cf68-42e4-8150-7a597239381a,Activation,410063007400690076006100740069006F006E0000000000
114,55,10458370,11557,2023-07-14 08:46:34 -05:00,2023-07-14,7.205759e+16,16818855,NaN,Application Hang,{00000000-0000-0000-0000-00000000000,...,10.0.19041.2193 (WinBuild.160101.0800),LockApp.exe,10.0.19041.2193,4264,01d9b5a19d58a49c,4294967295,C:\Windows\SystemApps\Microsoft.LockApp_cw5n1h...,293edfd9-0ea1-4efd-9e0a-8453aa65b4df,Cross-thread,430072006F00730073002D007400680072006500610064...
115,56,10458420,11557,2023-07-14 08:46:34 -05:00,2023-07-14,7.205759e+16,16816883,NaN,Application Hang,{00000000-0000-0000-0000-00000000000,...,10.0.19041.1 (WinBuild.160101.0800),svchost.exe,10.0.19041.1806,1594,01d9b4b1d082737b,4294967295,C:\Windows\System32\svchost.exe,2f4b2827-bb78-4a28-ac87-5b74d824fa10,Cross-thread,430072006F00730073002D007400680072006500610064...
